# Communicating with servers via HTTP

## `requests`

`requests` is a third-party package that faciliates the task of sending HTTP requests to communicate with servers.

- To install `requests` with `conda` run:

```bash
conda install requests
```

- To install `requests` with `pip` run:
```bash
pip install requests
```

You can try installing packages using the **Terminal** (Mac), **Anaconda Prompt** (Windows), or by launching a **Terminal** from **Jupyter notebook** (`New` => `Terminal` from the Jupyter notebook `Files` interface).

Whilst `requests` can be used to retrieve HTML from web pages, it shines when it comes to communicating with web APIs that require several arguments.

Let's first import `requests` and retrieve a web page.

We can call `get()` with an URL and save the `Response` object.

In [ ]:
import requests

# Get the response from a page
response = requests.get("https://cambridgespark.com")

Let's check the status code, `200` means `OK` -- the request was successful.

In [ ]:
response.status_code

We can also access the content of the page with the attribute `.content`

In [ ]:
response.content

Let's look at how `requests` helps when we work with more complex web APIs.

We've created our own web API, you can access it here [here](https://europe-west2-kate-dev.cloudfunctions.net/banking)

We define below the base URL for the API so you do not have to type the whole URL everytime and can just append the endpoint you want to call.

In [ ]:
BASE_URL = "https://europe-west2-kate-dev.cloudfunctions.net/banking"

Have a look at the [API Documentation](https://europe-west2-kate-dev.cloudfunctions.net/banking/documentation) to know more about what functionalities are available.

To start with, we will try to list all available users.

To do so, we can call the `/api/users` endpoint.

Let's call our first endpoint to get all available users:

In [ ]:
url_endpoint = f"{BASE_URL}/api/users"

r = requests.get(url=url_endpoint)

In [ ]:
# Let's verify we got a status code 200 first
r.status_code

Oops, error [401](https://httpstatuses.com/401) stands for `Unauthorized` - HTTP is a well defined protocol where each error corresponds to a specific issue. 

We can check the content returned by the API for more details:

In [ ]:
r.content

We "forgot" to mention that our API requires authentication... (see documentation for more details on how to authenticate)

Users need to pass an API key in the headers to authenticate, you can see it as a password. We provide the API key below.

Note: with actual APIs, the documentation will explain how to pass a key to authenticate, it often follows a similar process we are using here.

Thanksfully `requests` allows us to easily define the headers we want to pass when making a request:

In [ ]:
api_key = {"Authorization": "NRCqpfD3"}

Let's try again, with our API key this time

In [ ]:
url_endpoint = f"{BASE_URL}/api/users"

r = requests.get(url=url_endpoint, headers=api_key)

r.status_code

Status code is [200](https://httpstatuses.com/200), great!

Let's see the content:

In [ ]:
r.content

Since our API return JSON objects, we can now use `.json()` directly from `requests` to load the data as a dictionary.

In [ ]:
users = r.json()
users

### Exercise

First we will get transactions for a specific user - as you can see in the documentation, you can use the `get_transactions` endpoint here and eppend the user_id of the user you want to retrieve data from

Get transaction data for our first user (check the endpoint in the documentation, and don't forget the API key)

In [ ]:
user_id = 1
url_endpoint = f"{BASE_URL}/api/get_transactions/{user_id}"
r = requests.get(url=url_endpoint, headers=api_key)
r.json()


As you can see in the documentation, you can add an optional parameter to your request. With APIs you usually add a `?` followed by the optional arguments you want to add:

In [ ]:
user_id = 1
url_endpoint = f"{BASE_URL}/api/get_transactions/{user_id}?type=CREDIT"

r = requests.get(url=url_endpoint, headers=api_key)
r.json()

This can quickly become harder to work with as you are adding more optional parameters. `requests` provides a better way to add such parameters.

### Exercise

Instead of using `?` followed by the parameters, create a new dictionary with the arguments you want to use and their value, then pass this dictionary to `get()` as keyword argument `params`.

Syntax:

```
requests.get(your_url, params=your_parameters, headers=your_headers)
```

In [ ]:
user_id = 1
url_endpoint = f"{BASE_URL}/api/get_transactions/{user_id}"

# Our optional arguments
params = {"type": "CREDIT"}

r = requests.get(url_endpoint, params=params, headers=api_key)
r.json()


Our web API also supports some `POST` operations where you can add data to our database. The first one we will see here is `add_user` that allows you to add a new user.

With `requests` we can simply use the `.post()` method and pass the data we want to send as a `data` parameter.

Replace the name in the dictionary below by your own name. This defines the data about a user that we want to send to the API.

In [ ]:
# Replace by a name of your choice
user = {"name": "John"}

In [ ]:
# Here we have a new argument, data, that allows us to post data to the API
url_endpoint = f"{BASE_URL}/api/add_user"

r = requests.post(url_endpoint, data=user, headers=api_key)

Let's check the status code:

In [ ]:
r.status_code

`200` means the operation was successful! If we call the get_users endpoint again (like we did at the very beginning) we should see our new user.

Get the updated list of users to find your new user's ID:

In [ ]:
url_endpoint = f"{BASE_URL}/api/users"
r = requests.get(url=url_endpoint, headers=api_key)
r.json()


Finally, our web API allows us to add transactions by sending data serialised in json through a `POST` request to the same endpoint we've seen before. With `requests` we can simply use the `.post()` method and pass the data we want to send as a `data` parameter.

In [ ]:
# Same endpoint as before, but here we'll use a POST request
user_id = 2 
url_endpoint = f"{BASE_URL}/api/add_transaction/{user_id}"

# The advantage of using our own API is that we can credit accounts as we want :)
transaction_to_add = {"type": "CREDIT", "amount": 1000}

r = requests.post(url_endpoint, data=transaction_to_add, headers=api_key)

In [ ]:
# We verify the status code
r.status_code

### Exercise

Let's check that our transaction was added; call the endpoint to `get_transactions` for the account you credited.

- use a `.get()` request with the necessary `headers` and the same `user_id` as used above

In [ ]:
url_endpoint = "/".join([BASE_URL, "api/get_transactions", "2"])

r = requests.get(url_endpoint, headers=api_key)
r.json()


### [Optional] A more advanced exercise

By now you should have all the tools you need to write your own programmes that leverage the power of APIs and automate tasks.

For instance here you should be able to write a simple programme that compute the balance for a given user.

For this you will need to:
- retrieve all CREDIT transactions for a user
- compute the sum of credits
- retrieve all DEBIT transactions for a user
- compute the sum of debit
- compute the balance

In [ ]:
# Add your code below to compute the balance for a given user!

user_id = 1
url_endpoint = "/".join([BASE_URL, "api/get_transactions", "1"])

# compute total credit
params = {"type": "CREDIT"}
r = requests.get(url_endpoint, headers=api_key, params=params)
sum_credit = sum([float(c["amount"]) for c in r.json()])

# compute total debit
params = {"type": "DEBIT"}
r = requests.get(url_endpoint, headers=api_key, params=params)
sum_debit = sum([float(c["amount"]) for c in r.json()])

balance = sum_credit - sum_debit
